# 🧠 Fine-Tune GPT2 untuk Chatbot Kampus
Model ini akan dilatih dengan dataset tanya-jawab kampus berbasis GPT2.

In [1]:
!pip install transformers datasets

^C


   ---------------------------------------- 0.0/25.8 MB ? eta -:--:--
   ---------------------------------------- 0.3/25.8 MB ? eta -:--:--
    --------------------------------------- 0.5/25.8 MB 1.4 MB/s eta 0:00:19
   - -------------------------------------- 1.0/25.8 MB 1.7 MB/s eta 0:00:15
   -- ------------------------------------- 1.3/25.8 MB 1.8 MB/s eta 0:00:14
   -- ------------------------------------- 1.8/25.8 MB 1.8 MB/s eta 0:00:14
   --- ------------------------------------ 2.4/25.8 MB 1.9 MB/s eta 0:00:13
   ---- ----------------------------------- 2.6/25.8 MB 1.9 MB/s eta 0:00:13
   ---- ----------------------------------- 3.1/25.8 MB 1.9 MB/s eta 0:00:12
   ----- ---------------------------------- 3.4/25.8 MB 1.9 MB/s eta 0:00:12
   ------ --------------------------------- 3.9/25.8 MB 1.9 MB/s eta 0:00:12
   ------ --------------------------------- 4.2/25.8 MB 1.9 MB/s eta 0:00:12
   ------- -------------------------------- 4.7/25.8 MB 1.9 MB/s eta 0:00:11
   ------- --


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## 📁 Upload File Dataset `qa_kampus.jsonl`

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="qa_kampus.jsonl")
dataset = dataset["train"].train_test_split(test_size=0.1)
dataset

## 🔧 Load Tokenizer & Model GPT2

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

model = GPT2LMHeadModel.from_pretrained("gpt2")

## 🔠 Tokenisasi Dataset

In [ ]:
def tokenize(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=128)

tokenized_dataset = dataset.map(tokenize, batched=True)

## 🏋️ Fine-Tune GPT2

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    logging_steps=10,
    fp16=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer
)

trainer.train()

## 💾 Simpan Model & Tokenizer

In [ ]:
trainer.save_model("gpt2-kampus")
tokenizer.save_pretrained("gpt2-kampus")

## ⬇️ Download Model ke Laptop

In [ ]:
!zip -r gpt2-kampus.zip gpt2-kampus
files.download("gpt2-kampus.zip")